## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline to Extract neurologic deficits related to Stroke Scale (NIHSS)


- **Model**: `en.med_ner.nihss_pipeline`
- **Model Description**: The National Institutes of Health Stroke Scale (NIHSS) is a 15-item neurologic examination stroke scale. This pretrained pipeline is built on top of the `ner_nihss` model. It quantifies the physical manifestations of neurological deficits and provides crucial support for clinical decision-making and early-stage emergency triage.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "en-med-ner-nihss-pipeline"

content_type = "application/json"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3


# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data_dicts):
    for data_dict in data_dicts:
        json_input_data = json.dumps(data_dict)
        i = 1
        input_file_name = f'inputs/real-time/input{i}.json'
        output_file_name = f'outputs/real-time/out{i}.out'

        while os.path.exists(input_file_name) or os.path.exists(output_file_name):
            i += 1
            input_file_name = f'inputs/real-time/input{i}.json'
            output_file_name = f'outputs/real-time/out{i}.out'

        os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
        os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

        with open(input_file_name, 'w') as f:
            f.write(json_input_data)

        s3_client.put_object(Bucket=s3_bucket, Key=f"{model_name}/validation-input-json/real-time/{os.path.basename(input_file_name)}", Body=bytes(json_input_data.encode('UTF-8')))

        response = sm_runtime.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept="application/json",
            Body=json_input_data,
        )

        # Process response
        response_data = json.loads(response["Body"].read().decode("utf-8"))
        df = pd.DataFrame(response_data['predictions'])
        display(df)

        # Save response data to file
        with open(output_file_name, 'w') as f_out:
            json.dump(response_data, f_out, indent=4)


### Initial Setup

In [7]:
docs = [
    "BS soft , nontender ext : no edema NIH Stroke scale 15 .   1a LOC , Neurologic examination : Mental status : Awake and mildly lethargic , cooperative with most of exam , flat affect . Oriented to person . Mildly inattentive with exam . In the ED he was found to have L arm and leg weakness . He had a NIH stroke scale of 15 . He received tPA at 0155 which placed him.",
    "There , her initial NIHSS score was 4 , as recorded by the ED physicians . This included 2 for weakness in her left leg and 2 for what they felt was subtle ataxia in her left arm and leg .",

]

sample_text = "Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , 1to2 for gaze , two for face , eight for motor , one for limited ataxia , 1 to2 for sensory , three for best language . On the neurologic examination the patient was intermittently"

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [8]:
data_dicts = [
    {
        "text": sample_text
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)


,document,ner_chunk,begin,end,ner_label,confidence
0,"Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , 1to2 for gaze , two for face , eight for motor , one for limited ataxia , 1 to2 for sensory , three for best language . On the neurologic examination the patient was intermittently",NIH stroke scale,29,44,NIHSS,0.9729667
1,"Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , 1to2 for gaze , two for face , eight for motor , one for limited ataxia , 1 to2 for sensory , three for best language . On the neurologic examination the patient was intermittently",23 to 24,66,73,Measurement,0.86083335
2,"Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , 1to2 for gaze , two for face , eight for motor , one for limited ataxia , 1 to2 for sensory , three for best language . On the neurologic examination the patient was intermittently",one,81,83,Measurement,0.8693
3,"Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , 1to2 for gaze , two for face , eight for motor , one for limited ataxia , 1 to2 for sensory , three for best language . On the neurologic examination the patient was intermittently",consciousness,89,101,1a_LOC,0.6258
4,"Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , 1to2 for gaze , two for face , eight for motor , one for limited ataxia , 1 to2 for sensory , three for best language . On the neurologic examination the patient was intermittently",two,105,107,Measurement,0.9637
5,"Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , 1to2 for gaze , two for face , eight for motor , one for limited ataxia , 1 to2 for sensory , three for best language . On the neurologic examination the patient was intermittently",month and year,113,126,1b_LOCQuestions,0.8323667
6,"Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , 1to2 for gaze , two for face , eight for motor , one for limited ataxia , 1 to2 for sensory , three for best language . On the neurologic examination the patient was intermittently",two,132,134,Measurement,0.9711
7,"Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , 1to2 for gaze , two for face , eight for motor , one for limited ataxia , 1 to2 for sensory , three for best language . On the neurologic examination the patient was intermittently",eye / grip,140,149,1c_LOCCommands,0.9123667
8,"Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , 1to2 for gaze , two for face , eight for motor , one for limited ataxia , 1 to2 for sensory , three for best language . On the neurologic examination the patient was intermittently",1to2,153,156,Measurement,0.8095
9,"Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , 1to2 for gaze , two for face , eight for motor , one for limited ataxia , 1 to2 for sensory , three for best language . On the neurologic examination the patient was intermittently",gaze,162,165,2_BestGaze,0.9009


#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [9]:
data_dicts = [
    {
        "text": docs
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,ner_chunk,begin,end,ner_label,confidence
0,"BS soft , nontender ext : no edema NIH Stroke scale 15 . 1a LOC , Neurologic examination : Mental status : Awake and mildly lethargic , cooperative with most of exam , flat affect . Oriented to person . Mildly inattentive with exam . In the ED he was found to have L arm and leg weakness . He had a NIH stroke scale of 15 . He received tPA at 0155 which placed him.",NIH Stroke scale,35,50,NIHSS,0.9928667
1,"BS soft , nontender ext : no edema NIH Stroke scale 15 . 1a LOC , Neurologic examination : Mental status : Awake and mildly lethargic , cooperative with most of exam , flat affect . Oriented to person . Mildly inattentive with exam . In the ED he was found to have L arm and leg weakness . He had a NIH stroke scale of 15 . He received tPA at 0155 which placed him.",15,52,53,Measurement,0.9969
2,"BS soft , nontender ext : no edema NIH Stroke scale 15 . 1a LOC , Neurologic examination : Mental status : Awake and mildly lethargic , cooperative with most of exam , flat affect . Oriented to person . Mildly inattentive with exam . In the ED he was found to have L arm and leg weakness . He had a NIH stroke scale of 15 . He received tPA at 0155 which placed him.",1a LOC,59,64,1a_LOC,0.99705
3,"BS soft , nontender ext : no edema NIH Stroke scale 15 . 1a LOC , Neurologic examination : Mental status : Awake and mildly lethargic , cooperative with most of exam , flat affect . Oriented to person . Mildly inattentive with exam . In the ED he was found to have L arm and leg weakness . He had a NIH stroke scale of 15 . He received tPA at 0155 which placed him.",NIH stroke scale,301,316,NIHSS,0.9732
4,"BS soft , nontender ext : no edema NIH Stroke scale 15 . 1a LOC , Neurologic examination : Mental status : Awake and mildly lethargic , cooperative with most of exam , flat affect . Oriented to person . Mildly inattentive with exam . In the ED he was found to have L arm and leg weakness . He had a NIH stroke scale of 15 . He received tPA at 0155 which placed him.",15,321,322,Measurement,0.9877
5,"There , her initial NIHSS score was 4 , as recorded by the ED physicians . This included 2 for weakness in her left leg and 2 for what they felt was subtle ataxia in her left arm and leg .",NIHSS score,20,30,NIHSS,0.7844
6,"There , her initial NIHSS score was 4 , as recorded by the ED physicians . This included 2 for weakness in her left leg and 2 for what they felt was subtle ataxia in her left arm and leg .",4,36,36,Measurement,0.9997
7,"There , her initial NIHSS score was 4 , as recorded by the ED physicians . This included 2 for weakness in her left leg and 2 for what they felt was subtle ataxia in her left arm and leg .",2,89,89,Measurement,0.9942
8,"There , her initial NIHSS score was 4 , as recorded by the ED physicians . This included 2 for weakness in her left leg and 2 for what they felt was subtle ataxia in her left arm and leg .",left leg,111,118,6a_LeftLeg,0.6089
9,"There , her initial NIHSS score was 4 , as recorded by the ED physicians . This included 2 for weakness in her left leg and 2 for what they felt was subtle ataxia in her left arm and leg .",2,124,124,Measurement,0.9998


### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [10]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [11]:
import os

validation_file_name = "input.json"

validation_input_path = f"s3://{s3_bucket}/{model_name}/validation-input-json/batch"
validation_output_path = f"s3://{s3_bucket}/{model_name}/validation-output-json/batch"

input_dir = 'inputs/batch'
output_dir = 'outputs/batch'

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [12]:
import json

def write_and_upload_to_s3(json_input_data, file_name):

    json_data = json.dumps(json_input_data)

    with open(file_name, "w") as f:
        f.write(json_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input-json/batch/{os.path.basename(file_name)}",
        Body=(bytes(json_data.encode("UTF-8"))),
    )


In [13]:
# Define input JSON data for each validation file
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{input_dir}/{validation_file_name}")

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

In [15]:
from urllib.parse import urlparse

def process_s3_output_and_save(validation_file_name, output_file_name):

    output_file_path = f"{output_dir}/{output_file_name}"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data["predictions"])
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [16]:
process_s3_output_and_save(validation_file_name, "out.out")

,document,ner_chunk,begin,end,ner_label,confidence
0,"BS soft , nontender ext : no edema NIH Stroke scale 15 . 1a LOC , Neurologic examination : Mental status : Awake and mildly lethargic , cooperative with most of exam , flat affect . Oriented to person . Mildly inattentive with exam . In the ED he was found to have L arm and leg weakness . He had a NIH stroke scale of 15 . He received tPA at 0155 which placed him.",NIH Stroke scale,35,50,NIHSS,0.9928667
1,"BS soft , nontender ext : no edema NIH Stroke scale 15 . 1a LOC , Neurologic examination : Mental status : Awake and mildly lethargic , cooperative with most of exam , flat affect . Oriented to person . Mildly inattentive with exam . In the ED he was found to have L arm and leg weakness . He had a NIH stroke scale of 15 . He received tPA at 0155 which placed him.",15,52,53,Measurement,0.9969
2,"BS soft , nontender ext : no edema NIH Stroke scale 15 . 1a LOC , Neurologic examination : Mental status : Awake and mildly lethargic , cooperative with most of exam , flat affect . Oriented to person . Mildly inattentive with exam . In the ED he was found to have L arm and leg weakness . He had a NIH stroke scale of 15 . He received tPA at 0155 which placed him.",1a LOC,59,64,1a_LOC,0.99705
3,"BS soft , nontender ext : no edema NIH Stroke scale 15 . 1a LOC , Neurologic examination : Mental status : Awake and mildly lethargic , cooperative with most of exam , flat affect . Oriented to person . Mildly inattentive with exam . In the ED he was found to have L arm and leg weakness . He had a NIH stroke scale of 15 . He received tPA at 0155 which placed him.",NIH stroke scale,301,316,NIHSS,0.9732
4,"BS soft , nontender ext : no edema NIH Stroke scale 15 . 1a LOC , Neurologic examination : Mental status : Awake and mildly lethargic , cooperative with most of exam , flat affect . Oriented to person . Mildly inattentive with exam . In the ED he was found to have L arm and leg weakness . He had a NIH stroke scale of 15 . He received tPA at 0155 which placed him.",15,321,322,Measurement,0.9877
5,"There , her initial NIHSS score was 4 , as recorded by the ED physicians . This included 2 for weakness in her left leg and 2 for what they felt was subtle ataxia in her left arm and leg .",NIHSS score,20,30,NIHSS,0.7844
6,"There , her initial NIHSS score was 4 , as recorded by the ED physicians . This included 2 for weakness in her left leg and 2 for what they felt was subtle ataxia in her left arm and leg .",4,36,36,Measurement,0.9997
7,"There , her initial NIHSS score was 4 , as recorded by the ED physicians . This included 2 for weakness in her left leg and 2 for what they felt was subtle ataxia in her left arm and leg .",2,89,89,Measurement,0.9942
8,"There , her initial NIHSS score was 4 , as recorded by the ED physicians . This included 2 for weakness in her left leg and 2 for what they felt was subtle ataxia in her left arm and leg .",left leg,111,118,6a_LeftLeg,0.6089
9,"There , her initial NIHSS score was 4 , as recorded by the ED physicians . This included 2 for weakness in her left leg and 2 for what they felt was subtle ataxia in her left arm and leg .",2,124,124,Measurement,0.9998


In [17]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-med-ner-nihss-pipeline-2024-03-21-08-43-10-228


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

